In [90]:
import numpy as np
import pandas as pd

# easy way to split data set into train/dev sets
from sklearn.model_selection import train_test_split

# for generation a confusion matrix
from sklearn.metrics import confusion_matrix

from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.optimizers import Adam
from keras.utils import to_categorical

In [91]:
def extract_data(data):
  # reshape the bands from flattened (5625,1) vectors to (75,75) vectors
  # note that all elements are floats
  band_1 = np.array([np.array(band).reshape(75, 75).astype("float32") for band in data["band_1"]])
  band_2 = np.array([np.array(band).reshape(75, 75).astype("float32") for band in data["band_2"]])
  is_iceberg = data["is_iceberg"]

  features = np.concatenate((
    band_1[:, :, :, np.newaxis],
    band_2[:, :, :, np.newaxis]
  ), axis=-1)
  labels = is_iceberg

  return features, labels

In [92]:
def build_model():
  model = Sequential()

  # input layer
  model.add(Conv2D(64, kernel_size=(3, 3), activation="relu", input_shape=(75, 75, 2)))
  model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
  model.add(Dropout(0.2))

  # conv layer 1
  model.add(Conv2D(128, kernel_size=(3, 3), activation="relu"))
  model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
  model.add(Dropout(0.2))

  # conv layer 2
  model.add(Conv2D(64, kernel_size=(3, 3), activation="relu"))
  model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
  model.add(Dropout(0.2))

  # flatten layers
  model.add(Flatten())

  # dense layer
  model.add(Dense(256, activation="relu"))
  model.add(Dropout(0.2))

  # dense layer
  model.add(Dense(128, activation="relu"))
  model.add(Dropout(0.2))

  # sigmoid layer
  model.add(Dense(1, activation="sigmoid"))

  # compile model
  optimizer = Adam()
  model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

  # summary and return
  model.summary()
  return model

A label of 0 represents an iceberg while a label of 1 represents a not iceberg. We'll do some processing to help us pull an iceberg together.

In [93]:
import plotly.offline as py
import plotly.graph_objs as go

py.init_notebook_mode(connected=True)

def visualize_image(raw_data, title):
    data = [
        go.Surface(
            z=raw_data
        )
    ]
    layout = go.Layout(
        title=title,
        autosize=False,
        width=500,
        height=500,
        margin=dict(
            l=65,
            r=50,
            b=65,
            t=90
        )
    )
    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig, filename=title)

In [94]:
def data_analysis(band=0, label=0, selected_index=-1):
    raw_train = pd.read_json("data/train.json")
    raw_train_features, raw_train_labels = extract_data(raw_train)
    
    num_samples = raw_train_labels.shape[0]
    
    if (selected_index == -1):
        selected_index = np.random.randint(0, num_samples)
        while (raw_train_labels[selected_index] != label):
            selected_index = np.random.randint(0, num_samples)
    
    # get the data for the selected index
    raw_data = raw_train_features[selected_index]

    # draw out the image
    title = "Iceberg" if label == 1 else "Ship"
    visualize_image(raw_data[:,:,band], title + ": ID " + str(selected_index))

In [95]:
# grab some visualizations for a ship, band 0
data_analysis(1, 0, 1025)

In [96]:
# grab some visualizations for a iceberg, band 1
data_analysis(1, 1, 1238)

In [97]:
def train_nn(num_epochs):
  raw_train = pd.read_json("data/train.json")
  raw_train_features, raw_train_labels = extract_data(raw_train)

  # split into train and dev sets
  train_features, dev_features, train_labels, dev_labels = train_test_split(
    raw_train_features, raw_train_labels, random_state=0, train_size=0.8)
  
  # construct model to use
  model = build_model()

  # fit against the model
  model.fit(train_features, train_labels,
    batch_size=24, epochs=num_epochs, verbose=1,
    validation_data=(dev_features, dev_labels))

  # return the model
  return model

In [98]:
def test_nn(model):
    raw_test = pd.read_json("data/test.json")
    test_features, test_labels = extract_data(raw_test)
    
    # feed into model
    results = model.predict(test_features, test_labels)
    
    print(results.shape)

In [99]:
nn50model = train_nn(50)

/home/rogerchen/Development/kaggle-statoil/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning:

From version 0.21, test_size will always complement train_size unless both are specified.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 73, 73, 64)        1216      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 36, 36, 64)        0         
_________________________________________________________________
dropout_21 (Dropout)         (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 17, 17, 128)       0         
_________________________________________________________________
dropout_22 (Dropout)         (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 15, 15, 64)        73792     
__________

1283/1283 [==============================] - 2s 2ms/step - loss: 7.4372 - acc: 0.5386 - val_loss: 8.0842 - val_acc: 0.4984
Epoch 45/50
1283/1283 [==============================] - 2s 2ms/step - loss: 7.4372 - acc: 0.5386 - val_loss: 8.0842 - val_acc: 0.4984
Epoch 46/50
1283/1283 [==============================] - 2s 2ms/step - loss: 7.4372 - acc: 0.5386 - val_loss: 8.0842 - val_acc: 0.4984
Epoch 47/50
1283/1283 [==============================] - 2s 2ms/step - loss: 7.4372 - acc: 0.5386 - val_loss: 8.0842 - val_acc: 0.4984
Epoch 48/50
1283/1283 [==============================] - 2s 2ms/step - loss: 7.4372 - acc: 0.5386 - val_loss: 8.0842 - val_acc: 0.4984
Epoch 49/50
1283/1283 [==============================] - 2s 2ms/step - loss: 7.4372 - acc: 0.5386 - val_loss: 8.0842 - val_acc: 0.4984
Epoch 50/50
1283/1283 [==============================] - 2s 2ms/step - loss: 7.4372 - acc: 0.5386 - val_loss: 8.0842 - val_acc: 0.4984


In [100]:
test_nn(nn50model)

ValueError: Could not reserve memory block